In [1]:
import os

In [15]:
dataset_directory = "../Dataset/facebook"
if dataset_directory[-1] != '/':
    dataset_directory = dataset_directory + '/'
all_files = os.listdir(dataset_directory)
egofeat_files = [file for file in all_files if (file.split('.')[1] == 'egofeat')]

In [16]:
ego_nodes = [int(file.split('.')[0]) for file in egofeat_files]
ego_nodes

[0, 107, 1684, 1912, 3437, 348, 3980, 414, 686, 698]

In [17]:
nodes = {}
for ego_node in ego_nodes:
    nodes[ego_node] = set()
    circles = open(dataset_directory + str(ego_node) + '.circles', 'r')
    for line in circles:
        line = line.split()
        for n in line[1:]:
            nodes[ego_node].add(int(n))
    circles.close()
    edges = open(dataset_directory + str(ego_node) + '.edges', 'r')
    for line in edges:
        line = line.split()
        for n in line:
            nodes[ego_node].add(int(n))
    edges.close()

In [18]:
edges_list = {}
for ego_node in ego_nodes:
    edge_list = set()
    
    circles = open(dataset_directory + str(ego_node) + '.circles', 'r')
    for line in circles:
        line = line.split()
        for n in line[1:]:
            edge_list.add((min(ego_node, int(n)), max(ego_node, int(n))))
    circles.close()
    
    edges = open(dataset_directory + str(ego_node) + '.edges', 'r')
    for line in edges:
        line = line.split()
        one = int(line[0])
        two = int(line[1])
        edge_list.add((min(one, two), max(one, two)))        
    edges.close()
    edges_list[ego_node] = edge_list

In [19]:
import random

In [20]:
try:
    os.mkdir('../Edges/')
except:
    pass
for ego_node in ego_nodes:
    edges = list(edges_list[ego_node])
    random.shuffle(edges)
    size = len(edges)
    for i in range(5):
        edges_20 = edges[i * (size // 5):(i + 1) * (size // 5)]
        edges_80 = edges[:i * (size // 5)] + edges[(i + 1) * (size // 5):]
        
        file = open('../Edges/edges_20_{}_{}.txt'.format(ego_node, i), 'w')
        for edge in edges_20:
            file.write('{} {}\n'.format(edge[0], edge[1]))
        file.close()
        
        file = open('../Edges/edges_80_{}_{}.txt'.format(ego_node, i), 'w')
        for edge in edges_80:
            file.write('{} {}\n'.format(edge[0], edge[1]))
        file.close()

In [31]:
try:
    os.mkdir('../Adjacency_Lists/')
except:
    pass
for ego_node in ego_nodes:
    for i in range(5):
        file = open('../Edges/edges_80_{}_{}.txt'.format(ego_node, i), 'r')
        adjacency_list = {}
        for line in file:
            if len(line) == 0:
                continue
            line = line.split()
            node1, node2 = int(line[0]), int(line[1])
            if node1 not in adjacency_list.keys():
                adjacency_list[node1] = set()
            if node2 not in adjacency_list.keys():
                adjacency_list[node2] = set()
            adjacency_list[node1].add(node2)
            adjacency_list[node2].add(node1)
        file.close()
        keys = list(adjacency_list.keys())
        keys.sort()

        file = open('../Adjacency_Lists/adjacency_list_{}_{}.txt'.format(ego_node, i), 'w')
        for key in keys:
            file.write('{}'.format(key))
            for node in adjacency_list[key]:
                file.write(' {}'.format(node))
            file.write('\n')
        file.close()

In [33]:
features = {}
gender_featnum = []
for ego_node in ego_nodes:
    featnames = open(dataset_directory + str(ego_node) + '.featnames')
    features_node = []
    for line in featnames:
        features_node.append(int(line.split(';')[-1].split()[-1]))
        if line.split(';')[0].split()[1] == 'gender':
            if int(line.split(';')[-1].split()[-1]) not in gender_featnum:
                gender_featnum.append(int(line.split(';')[-1].split()[-1]))
    features[ego_node] = features_node
    featnames.close()
gender_featnum.sort()

In [34]:
gender_1 = {}
gender_2 = {}
for ego_node in ego_nodes:
    egofeat = open(dataset_directory + str(ego_node) + '.egofeat')
    feat = open(dataset_directory + str(ego_node) + '.feat')
    index_gender_1 = features[ego_node].index(gender_featnum[0])
    index_gender_2 = features[ego_node].index(gender_featnum[1])
    for line in egofeat:
        if int(line.split()[index_gender_1]) == 1:
            gender_1[ego_node] = 1
        else:
            gender_1[ego_node] = 0
        if int(line.split()[index_gender_2]) == 1:
            gender_2[ego_node] = 1
        else:
            gender_2[ego_node] = 0
    for line in feat:
        if int(line.split()[index_gender_1 + 1]) == 1:
            gender_1[int(line.split()[0])] = 1
        else:
            gender_1[int(line.split()[0])] = 0
        if int(line.split()[index_gender_2 + 1]) == 1:
            gender_2[int(line.split()[0])] = 1
        else:
            gender_2[int(line.split()[0])] = 0

In [40]:
try:
    os.mkdir('../Gender_Adjacency_Lists/')
except:
    pass
for ego_node in ego_nodes:
    for i in range(5):
        gender_wise_adjacency_list = {}
        file = open('../Adjacency_Lists/adjacency_list_{}_{}.txt'.format(ego_node, i), 'r')
        for line in file:
            if len(line) == 0:
                continue
            line = line.split()
            gender_wise_adjacency_list[int(line[0])] = []
            gender_1_list = [int(n) for n in line[1:] if gender_1[int(n)] == 1]
            gender_2_list = [int(n) for n in line[1:] if gender_2[int(n)] == 1]
            gender_3_list = [int(n) for n in line[1:] if (gender_1[int(n)] == 0 and gender_2[int(n)] == 0)]
            gender_wise_adjacency_list[int(line[0])].append(gender_1_list)
            gender_wise_adjacency_list[int(line[0])].append(gender_2_list)
            gender_wise_adjacency_list[int(line[0])].append(gender_3_list)
        file.close()
        keys = list(gender_wise_adjacency_list.keys())
        keys.sort()
        file = open('../Gender_Adjacency_Lists/gender_wise_adjacency_list_{}_{}.txt'.format(ego_node, i), 'w')
        for key in keys:
            file.write('{}\n'.format(key))
            for elements in gender_wise_adjacency_list[key]:
                line = ''
                for element in elements:
                    line = line + str(element) + ' '
                file.write(line.strip() + '\n')

In [53]:
try:
    os.mkdir('../Fairwalks/')
except:
    pass
num_walks = 20
walk_len = 80
gender_choices = [0, 1, 2]
gender_choices_not_0 = [1, 2]
gender_choices_not_1 = [0, 2]
gender_choices_not_2 = [0, 1]

for ego_node in ego_nodes:
    for i in range(5):
        file = open('../Gender_Adjacency_Lists/gender_wise_adjacency_list_{}_{}.txt'.format(ego_node, i), 'r')
        gender_wise_adjacency_list = {}
        lines = file.readlines()
        j = 0
        while j < len(lines) and len(lines[j].strip()) != 0:
            node = int(lines[j].strip())
            gender_wise_adjacency_list[node] = []
            for k in range(3):
                j += 1
                gender_wise_adjacency_list[node].append([int(n) for n in lines[j].strip().split()])
            j += 1
        nodes = [int(n.strip()) for n in lines[::4]]
        file.close()
        file = open('../Fairwalks/fairwalks_{}_{}.txt'.format(ego_node, i), 'w')
        for walk in range(num_walks):
            for node in nodes:
                prev_node = -1
                trace = []
                current_node = node
                for covered_len in range(walk_len):
                    trace.append(current_node)
                    if len(gender_wise_adjacency_list[current_node][0]) == 0 and len(gender_wise_adjacency_list[current_node][1]) == 0:
                        gender_choice = 2
                    elif len(gender_wise_adjacency_list[current_node][0]) == 0 and len(gender_wise_adjacency_list[current_node][2]) == 0:
                        gender_choice = 1
                    elif len(gender_wise_adjacency_list[current_node][1]) == 0 and len(gender_wise_adjacency_list[current_node][2]) == 0:
                        gender_choice = 0
                    elif len(gender_wise_adjacency_list[current_node][2]) == 0:
                        gender_choice = random.choice(gender_choices_not_2)
                    elif len(gender_wise_adjacency_list[current_node][1]) == 0:
                        gender_choice = random.choice(gender_choices_not_1)
                    elif len(gender_wise_adjacency_list[current_node][0]) == 0:
                        gender_choice = random.choice(gender_choices_not_0)
                    else:
                        gender_choice = random.choice(gender_choices)
                    next_node = random.choice(gender_wise_adjacency_list[current_node][gender_choice])
                    prev_node = current_node
                    current_node = next_node
                line = ''
                for step in trace:
                    line = line + str(step) + ' '
                file.write(line.strip() + '\n')
                line = ''
                for step in trace[::-1]:
                    line = line + str(step) + ' '
                file.write(line.strip() + '\n')
        file.close()

In [54]:
from gensim.models import Word2Vec
import multiprocessing as mp

In [56]:
try:
    os.mkdir('../Embeddings')
except:
    pass

for ego_node in ego_nodes:
    for i in range(5):
        file = open('../Fairwalks/fairwalks_{}_{}.txt'.format(ego_node, i), 'r')
        walks = []
        lines = file.readlines()
        for line in lines:
            if len(line) == 0:
                continue
            walk = [element for element in line.strip().split()]
            walks.append(walk)
        model = Word2Vec(walks, size = 128, window = 10, min_count = 10, sg = 1, workers = mp.cpu_count())
        model.wv.save_word2vec_format('../Embeddings/embeddings_{}_{}.emb'.format(ego_node, i))

In [ ]:
import numpy as np

In [ ]:
for ego_node in ego_nodes:
    file = open('../Embeddings/' + str(ego_node) + '.emb')
    lines = file.readlines()
    num_nodes, ndims = int(lines[0].strip().split()[0]),int(lines[0].strip().split()[1])

    embeddings = {}
    for line in lines[1:]:
        line = line.strip()
        if len(line) == 0:
            continue
        line = line.split()
        node = int(line[0])
        embeddings[node] = np.asarray([float(element) for element in line[1:]], dtype = np.float32)

    nodes = list(embeddings.keys())
    nodes.sort()

    hadamard_vectors = {}
    for i in range(len(nodes)):
        node1 = nodes[i]
        for node2 in nodes[i + 1:]:
            hadamard_vectors[(node1, node2)] = np.multiply(embeddings[node1], embeddings[node2])
        print(node1)

    file2 = open('../Embeddings/hadamard_' + str(ego_node) + '.txt', 'w')
    for i in range(len(nodes)):
        node1 = nodes[i]
        for node2 in nodes[i + 1:]:
            file2.write(str(node1) + ' ' + str(node2) + ' ')
            string = ''
            for element in hadamard_vectors[(node1, node2)]:
                string = string + str(element) + ' '
            file2.write(string)
            file2.write('\n')
    file2.close()

In [ ]:
hadamard_vectors

In [ ]:
nodes

In [ ]:
adjacency_lists[0][1]

In [ ]:
import numpy as np

In [ ]:
directory = '../Embeddings/'
ego_node = ego_nodes[0]
X_nodes = []
X = []
y = []
file = open(directory + 'hadamard_' + str(ego_node) + '.txt', 'r')
adjacency_list = adjacency_lists[ego_node]
lines = file.readlines()
for line in lines:
    if len(line) == 0:
        continue
    line = line.split()
    node1, node2 = int(line[0]), int(line[1])
    X_nodes.append([node1, node2])
    if node2 in adjacency_list[node1]:
        y.append(1)
    else:
        y.append(0)
    vectors = [float(element) for element in line[2:]]
    X.append(vectors)

X_nodes = np.array(X_nodes)
X = np.array(X, dtype = np.float32)
y = np.array(y, dtype = np.bool)

In [ ]:
X_nodes

In [ ]:
X

In [ ]:
y

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_X, y, test_size = 0.2)

In [ ]:
model = RandomForestClassifier()
model.fit(X_train, y_train)

In [ ]:
rf_labels = model.predict(X_test)
rf_probs = model.predict_proba(X_test)[:, 1]

In [ ]:
from sklearn.metrics import roc_auc_score

# Calculate roc auc
roc_value = roc_auc_score(y_test, rf_probs)

In [ ]:
roc_value

In [ ]:
true = len(rf_labels[rf_labels == True])

In [ ]:
true

In [ ]:
false = len(rf_labels[rf_labels == False])

In [ ]:
false

In [ ]:
11546 / (11546 + 185)